In [8]:
%reload_ext autoreload
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pipetools import pipe, utils, where
from fastcore.basics import typed
import requests
from fastcore.foundation import L
import pandas as pd
from typing import List
from pathlib import Path
import re
from datetime import datetime, date
from loguru import logger
import os

In [9]:
import sys
sys.path.append("../../")

In [10]:
from src.run import run_fbref
from src.fbref.utilities import format_age, get_proxy_format, to_float

In [81]:
#stats = run_fbref(debug=True)

In [12]:
stats_available = ["shooting_stats", "passing_stats", "extra_passing_stats", "gca_stats",
                   "defensive_action_stats", "possession_stats", "playing_time_stats", "misc_stats"]

In [22]:
passing_df = pd.DataFrame([player._asdict() for player in stats[0]['passing_stats']])

In [23]:
passing_df.head()

,name,passes_completed,passes,passes_pct,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,...,passes_long,passes_pct_long,assists,xa,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes
0,Andrew Robertson,1370,1732,79.1,25772.0,8468.0,570,649,87.8,634,...,294,51.7,5,4.0,+1.0,37,97,41,16,144
1,Roberto Firmino,507,639,79.3,7594.0,1693.0,295,351,84.0,161,...,42,73.8,3,2.7,+0.3,20,34,15,0,54
2,Georginio Wijnaldum,846,910,93.0,14690.0,2866.0,412,436,94.5,334,...,91,90.1,0,1.2,-1.2,14,75,16,1,49
3,Sadio Mané,431,587,73.4,7040.0,1492.0,226,285,79.3,154,...,60,66.7,2,2.7,-0.7,33,17,35,3,57
4,Fabinho,1184,1285,92.1,25182.0,6436.0,317,336,94.3,669,...,233,81.1,0,0.4,-0.4,3,56,3,1,52


In [25]:
passing_df.size

598

In [26]:
fbref_all = pd.DataFrame({})
for team_stat in stats:
    team_lvl = pd.DataFrame({})
    print(f"Formatting for {team_stat['team'].lower()}")
    for stats in stats_available:
        if team_lvl.size == 0:
            team_lvl = pd.DataFrame(team_stat[stats])
        else:
            team_lvl = pd.merge(team_lvl, pd.DataFrame(team_stat[stats]), on=['name'])
    fbref_all = pd.concat([fbref_all, team_lvl])

Formatting for liverpool
Formatting for aston_villa
Formatting for leeds_united
Formatting for crystal_palace
Formatting for chelsea
Formatting for leicester_city
Formatting for wolves
Formatting for tottenham_hotspurs
Formatting for westham_united
Formatting for manchester_city
Formatting for everton
Formatting for southampton
Formatting for newcastle_united
Formatting for manchester_united
Formatting for brighton
Formatting for westbrom
Formatting for burnley
Formatting for sheffield_united
Formatting for fulham
Formatting for arsenal


In [27]:
fbref_all.shape

(473, 162)

In [28]:
fbref_all.head()

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,crosses_y,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Andrew Robertson,https://fbref.com/en/players/2e4f5f03/Andrew-R...,"[sco, SCO]",DF,26 years and 320 days,19.0,1,10,3,30.0,...,95,10,10,0,1,0,218,19,17,52.8
1,Roberto Firmino,https://fbref.com/en/players/4c370d81/Roberto-...,"[br, BRA]",FW,29 years and 115 days,16.8,5,46,16,34.8,...,18,5,12,0,0,0,96,15,53,22.1
2,Georginio Wijnaldum,https://fbref.com/en/players/eb58eef0/Georgini...,"[nl, NED]",MF,30 years and 075 days,17.0,1,16,4,25.0,...,2,8,19,0,0,0,153,20,33,37.7
3,Sadio Mané,https://fbref.com/en/players/c691bfe2/Sadio-Mane,"[sn, SEN]",FW,28 years and 290 days,16.8,6,52,20,38.5,...,34,9,20,1,0,0,83,16,43,27.1
4,Fabinho,https://fbref.com/en/players/7f3b388c/Fabinho,"[br, BRA]","DF,MF",27 years and 094 days,16.4,0,4,0,0.0,...,1,23,28,1,1,0,194,39,31,55.7


In [29]:
FBREF_DATA = Path("../../data/fbref-data")

In [30]:
date.today()

datetime.date(2021, 1, 25)

In [31]:
t_now = re.sub(r'\D', '', str(date.today()))

In [32]:
LIVE_GW = 19

In [35]:
FBREF_DATA.mkdir(parents=True, exist_ok=True)
fbref_all.to_csv(FBREF_DATA / f"{t_now}_GW{LIVE_GW}_player_stats.csv", index=False)

## FPL Data

In [36]:
fpl_request = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")

In [37]:
all_fpl_data = fpl_request.content

In [38]:
all_fpl_data = json.loads(all_fpl_data)

In [39]:
all_fpl_data.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [40]:
all_fpl_data["total_players"]

7885616

In [41]:
fpl_players = L(all_fpl_data["elements"])

In [42]:
fpl_teams = L(all_fpl_data["teams"])

In [43]:
fpl_teams[0]

{'code': 3,
 'draw': 0,
 'form': None,
 'id': 1,
 'loss': 0,
 'name': 'Arsenal',
 'played': 0,
 'points': 0,
 'position': 0,
 'short_name': 'ARS',
 'strength': 4,
 'team_division': None,
 'unavailable': False,
 'win': 0,
 'strength_overall_home': 1190,
 'strength_overall_away': 1210,
 'strength_attack_home': 1170,
 'strength_attack_away': 1210,
 'strength_defence_home': 1190,
 'strength_defence_away': 1200,
 'pulse_id': 1}

In [44]:
@typed
def extract_team_details(team: dict) -> dict:
    return {str(team["code"]): {"name": team["name"].lower().replace(" ", "_"), "slug": team["short_name"]}}

In [45]:
fpl_team_mapper = {}
[fpl_team_mapper.update(extract_team_details(team)) for team in fpl_teams]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [46]:
fpl_team_mapper

{'3': {'name': 'arsenal', 'slug': 'ARS'},
 '7': {'name': 'aston_villa', 'slug': 'AVL'},
 '36': {'name': 'brighton', 'slug': 'BHA'},
 '90': {'name': 'burnley', 'slug': 'BUR'},
 '8': {'name': 'chelsea', 'slug': 'CHE'},
 '31': {'name': 'crystal_palace', 'slug': 'CRY'},
 '11': {'name': 'everton', 'slug': 'EVE'},
 '54': {'name': 'fulham', 'slug': 'FUL'},
 '13': {'name': 'leicester', 'slug': 'LEI'},
 '2': {'name': 'leeds', 'slug': 'LEE'},
 '14': {'name': 'liverpool', 'slug': 'LIV'},
 '43': {'name': 'man_city', 'slug': 'MCI'},
 '1': {'name': 'man_utd', 'slug': 'MUN'},
 '4': {'name': 'newcastle', 'slug': 'NEW'},
 '49': {'name': 'sheffield_utd', 'slug': 'SHU'},
 '20': {'name': 'southampton', 'slug': 'SOU'},
 '6': {'name': 'spurs', 'slug': 'TOT'},
 '35': {'name': 'west_brom', 'slug': 'WBA'},
 '21': {'name': 'west_ham', 'slug': 'WHU'},
 '39': {'name': 'wolves', 'slug': 'WOL'}}

In [69]:
all_fpl_data["events"][-1]

{'id': 38,
 'name': 'Gameweek 38',
 'deadline_time': '2021-05-23T13:30:00Z',
 'average_entry_score': 0,
 'finished': False,
 'data_checked': False,
 'highest_scoring_entry': None,
 'deadline_time_epoch': 1621776600,
 'deadline_time_game_offset': 0,
 'highest_score': None,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'chip_plays': [],
 'most_selected': None,
 'most_transferred_in': None,
 'top_element': None,
 'top_element_info': None,
 'transfers_made': 0,
 'most_captained': None,
 'most_vice_captained': None}

In [73]:
gw_list = L([{"gw": event["name"], "finished": event["finished"]} for event in all_fpl_data["events"]])

In [80]:
gw_list.filter(lambda x: not x["finished"])[0]["gw"].split(" ")[1]

'20'

In [47]:
all_fpl_data["events"][0]

{'id': 1,
 'name': 'Gameweek 1',
 'deadline_time': '2020-09-12T10:00:00Z',
 'average_entry_score': 50,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 4761681,
 'deadline_time_epoch': 1599904800,
 'deadline_time_game_offset': 0,
 'highest_score': 142,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 112843},
  {'chip_name': '3xc', 'num_played': 225426}],
 'most_selected': 259,
 'most_transferred_in': 12,
 'top_element': 254,
 'top_element_info': {'id': 254, 'points': 20},
 'transfers_made': 0,
 'most_captained': 4,
 'most_vice_captained': 4}

In [48]:
all_fpl_data["phases"][3]

{'id': 4, 'name': 'November', 'start_event': 8, 'stop_event': 10}

In [49]:
category_types = {}
[category_types.update({str(player_cat["id"]): player_cat["singular_name_short"]}) for player_cat in all_fpl_data["element_types"]]

[None, None, None, None]

In [50]:
category_types

{'1': 'GKP', '2': 'DEF', '3': 'MID', '4': 'FWD'}

In [51]:
all_fpl_data["element_stats"]

[{'label': 'Minutes played', 'name': 'minutes'},
 {'label': 'Goals scored', 'name': 'goals_scored'},
 {'label': 'Assists', 'name': 'assists'},
 {'label': 'Clean sheets', 'name': 'clean_sheets'},
 {'label': 'Goals conceded', 'name': 'goals_conceded'},
 {'label': 'Own goals', 'name': 'own_goals'},
 {'label': 'Penalties saved', 'name': 'penalties_saved'},
 {'label': 'Penalties missed', 'name': 'penalties_missed'},
 {'label': 'Yellow cards', 'name': 'yellow_cards'},
 {'label': 'Red cards', 'name': 'red_cards'},
 {'label': 'Saves', 'name': 'saves'},
 {'label': 'Bonus', 'name': 'bonus'},
 {'label': 'Bonus Points System', 'name': 'bps'},
 {'label': 'Influence', 'name': 'influence'},
 {'label': 'Creativity', 'name': 'creativity'},
 {'label': 'Threat', 'name': 'threat'},
 {'label': 'ICT Index', 'name': 'ict_index'}]

In [52]:
@typed
def extract_player_fpl_stats(player: dict) -> dict:
    return {
        "player_id": player["id"],
        "name": f"{player['first_name']} {player['second_name']}",
        "display_slug": player["web_name"],
        "team_id": player["team"],
        "team": fpl_team_mapper.get(str(player["team_code"])).get("name"),
        "team_slug": fpl_team_mapper.get(str(player["team_code"])).get("slug"),
        "position": category_types.get(str(player["element_type"])),
        "points_per_game": to_float(player["points_per_game"]),
        "form": to_float(player["form"]),
        "news": player["news"],
        "news_added_at": player["news_added"],
        "status": player["status"],
        "total_points": player["total_points"],
        "value_form": to_float(player["value_form"]),
        "value_season": to_float(player["value_season"]),
        "selected_by_percent": to_float(player["selected_by_percent"]),
        "points_per_game": to_float(player["points_per_game"]),
        "influence_rank_overall": player["influence_rank"],
        "creativity_rank_overall": player["creativity_rank"],
        "threat_rank_overall": player["threat_rank"],
        "influence_rank_by_position": player["influence_rank_type"],
        "creativity_rank_by_position": player["creativity_rank_type"],
        "threat_rank_by_position:": player["threat_rank_type"],
        "ict_index": player["ict_index_rank"],
        "ict_index_by_position": player["ict_index_rank_type"],
        "dreamteam_count": player["dreamteam_count"],
        "transfers_in": player["transfers_in"],
        "transfers_out": player["transfers_out"],
        "chance_of_playing_next_round": player["chance_of_playing_next_round"],
        "chance_of_playing_this_round": player["chance_of_playing_this_round"],
        "cost_change_event": player["cost_change_event"],
        "cost_change_event_fall": player["cost_change_event_fall"],
        "cost_change_start": player["cost_change_start"],
        "cost_change_start_fall": player["cost_change_start_fall"],
        "ep_next": player["ep_next"],
        "ep_this": player["ep_this"],
        "now_cost": player["now_cost"],
        "transfers_in": player["transfers_in"],
        "transfers_out": player["transfers_out"],
        "transfers_in_event": player["transfers_in_event"],
        "transfers_out_event": player["transfers_out_event"],
        "goals_scored": player["goals_scored"],
        "assists": player["assists"],
        "clean_sheets": player["clean_sheets"],
        "goals_conceded": player["goals_conceded"],
        "penalties_saved": player["penalties_saved"],
        "penalties_missed": player["penalties_missed"],
        "saves": player["saves"],
        "bonus": player["bonus"],
        "bps": player["bps"],
        "corners_and_indirect_freekicks_order": player["corners_and_indirect_freekicks_order"],
        "corners_and_indirect_freekicks_text": player["corners_and_indirect_freekicks_text"],
        "direct_freekicks_order": player["direct_freekicks_order"],
        "direct_freekicks_text": player["direct_freekicks_text"],
        "penalties_order": player["penalties_order"],
        "penalties_text": player["penalties_text"],
    }

In [53]:
%time fpl_player_stats = L([extract_player_fpl_stats(player) for player in fpl_players])

CPU times: user 44 ms, sys: 40 µs, total: 44.1 ms
Wall time: 41.7 ms


In [54]:
fpl_player_stats_df = pd.DataFrame.from_dict(fpl_player_stats)

In [55]:
FPL_DATA = Path("../../data/fpl-data")

In [56]:
date.today()

datetime.date(2021, 1, 25)

In [57]:
t_now = re.sub(r'\D', '', str(date.today()))

#### Check gameweek aggregated stats

In [58]:
all_fpl_data["events"][1]

{'id': 2,
 'name': 'Gameweek 2',
 'deadline_time': '2020-09-19T10:00:00Z',
 'average_entry_score': 59,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 6234344,
 'deadline_time_epoch': 1600509600,
 'deadline_time_game_offset': 0,
 'highest_score': 165,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 94615},
  {'chip_name': 'freehit', 'num_played': 111968},
  {'chip_name': 'wildcard', 'num_played': 494000},
  {'chip_name': '3xc', 'num_played': 221133}],
 'most_selected': 259,
 'most_transferred_in': 302,
 'top_element': 390,
 'top_element_info': {'id': 390, 'points': 24},
 'transfers_made': 14637421,
 'most_captained': 4,
 'most_vice_captained': 254}

In [59]:
live_gameweek = 19

In [60]:
FPL_DATA.mkdir(parents=True, exist_ok=True)
fpl_player_stats_df.to_csv(FPL_DATA / f"{t_now}_GW{live_gameweek}_player_stats.csv", index=False)

### Merge fbref and fpl data

In [ ]:
fpl_player_stats_df.head()

In [ ]:
fpl_player_stats_df.groupby(["team_slug", "position"])["player_id"].count().reset_index().rename(
    columns={'player_id': 'count'})

In [61]:
fbref_all.head()

,name,profile_url,nationality,position,age,minutes_90s_x,goals,shots_total,shots_on_target,shots_on_target_pct,...,crosses_y,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Andrew Robertson,https://fbref.com/en/players/2e4f5f03/Andrew-R...,"[sco, SCO]",DF,26 years and 320 days,19.0,1,10,3,30.0,...,95,10,10,0,1,0,218,19,17,52.8
1,Roberto Firmino,https://fbref.com/en/players/4c370d81/Roberto-...,"[br, BRA]",FW,29 years and 115 days,16.8,5,46,16,34.8,...,18,5,12,0,0,0,96,15,53,22.1
2,Georginio Wijnaldum,https://fbref.com/en/players/eb58eef0/Georgini...,"[nl, NED]",MF,30 years and 075 days,17.0,1,16,4,25.0,...,2,8,19,0,0,0,153,20,33,37.7
3,Sadio Mané,https://fbref.com/en/players/c691bfe2/Sadio-Mane,"[sn, SEN]",FW,28 years and 290 days,16.8,6,52,20,38.5,...,34,9,20,1,0,0,83,16,43,27.1
4,Fabinho,https://fbref.com/en/players/7f3b388c/Fabinho,"[br, BRA]","DF,MF",27 years and 094 days,16.4,0,4,0,0.0,...,1,23,28,1,1,0,194,39,31,55.7


In [63]:
fpl_player_stats_df.head()

,player_id,name,display_slug,team_id,team,team_slug,position,points_per_game,form,news,...,penalties_missed,saves,bonus,bps,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,1,Mesut Özil,Özil,1,arsenal,ARS,MID,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,...,0,0,0,0,NaN,,NaN,,NaN,
1,2,Sokratis Papastathopoulos,Sokratis,1,arsenal,ARS,DEF,0.0,0.0,Left the club by mutual consent on 20/1,...,0,0,0,0,NaN,,NaN,,NaN,
2,3,David Luiz Moreira Marinho,David Luiz,1,arsenal,ARS,DEF,2.0,2.2,,...,0,0,0,108,NaN,,4.0,,NaN,
3,4,Pierre-Emerick Aubameyang,Aubameyang,1,arsenal,ARS,MID,4.2,4.8,Personal Reasons - 75% chance of playing,...,0,0,6,202,NaN,,NaN,,1.0,
4,5,Cédric Soares,Cédric,1,arsenal,ARS,DEF,6.5,2.4,,...,0,0,3,42,NaN,,NaN,,NaN,


In [64]:
fpl_player_stats_df[['player_id', 'name']]

,player_id,name
0,1,Mesut Özil
1,2,Sokratis Papastathopoulos
2,3,David Luiz Moreira Marinho
3,4,Pierre-Emerick Aubameyang
4,5,Cédric Soares
...,...,...
636,615,Lewis Richards
637,616,Christian Marques
638,617,Andreas Söndergaard
639,626,Patrick Cutrone


In [65]:
fbref_all[['name', 'profile_url']]

,name,profile_url
0,Andrew Robertson,https://fbref.com/en/players/2e4f5f03/Andrew-R...
1,Roberto Firmino,https://fbref.com/en/players/4c370d81/Roberto-...
2,Georginio Wijnaldum,https://fbref.com/en/players/eb58eef0/Georgini...
3,Sadio Mané,https://fbref.com/en/players/c691bfe2/Sadio-Mane
4,Fabinho,https://fbref.com/en/players/7f3b388c/Fabinho
...,...,...
20,Martinelli,https://fbref.com/en/players/48a5a5d6/Martinelli
21,Cédric Soares,https://fbref.com/en/players/839c14e1/Cedric-S...
22,Sead Kolašinac,https://fbref.com/en/players/3935e52e/Sead-Kol...
23,Reiss Nelson,https://fbref.com/en/players/c5bdb6e3/Reiss-Ne...
